In [373]:
#import modules for web scraping
from __future__ import print_function, division
import requests
from bs4 import BeautifulSoup

In [55]:
#get html code
url = 'https://www.boxofficemojo.com/alltime/domestic.htm'
response = requests.get(url)

In [56]:
#check if it worked
response.status_code

200

In [94]:
#make soup
page = response.text
soup_domestic = BeautifulSoup(page,"lxml")

In [95]:
print(soup_domestic.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="en">
 <head>
  <title>
   All Time Domestic Box Office Results
  </title>
  <meta content="all, time, domestic, box, office, result, gross, grosses, titanic, all, time, report, movie, film" name="keywords"/>
  <meta content="All time domestic box office results." name="description"/>
  <link charset="utf-8" href="/css/mojo.css?1" media="screen" rel="stylesheet" title="no title" type="text/css"/>
  <link charset="utf-8" href="/css/mojo.css?1" media="print" rel="stylesheet" title="no title" type="text/css"/>
 </head>
 <body>
  <iframe frameborder="0" height="1" id="sis_pixel_sitewide" marginheight="0" marginwidth="0" style="display: none;" width="1">
  </iframe>
  <script>
   setTimeout(function(){
        try{
            //sis3.0 pixel
            var cacheBust = Math.random() * 10000000000000000,
                url_sis3 = '//s.amazon-adsystem.com/iu3?',
                p

In [139]:
tables=soup_domestic.find_all("table")
rows=[row for row in tables[5].find_all('tr')]
columns = rows[0]
rows = rows[1:20]

# for row in rows:
#     print(row)
#     print()

movies = {}
for row in rows:
    test_items = row.find_all('td')
    test_title = test_items[1].find('a')['href']
    movies[test_title] = [i.text for i in test_items[1:]]

columns = columns.find_all('a')
columns = [column.text for column in columns][1:]

# print(movies)

# list(movies.keys())
# import defaultdict

# movies_dict_for_df = defaultdict()
# for key in movies:
#     movies[key][0] = 

In [254]:
#clean the title with regex as an exercise
import re

pattern = re.compile(r'[a-zA-z ]+')
# for column in columns:
#     pattern.search(column).group(0)
#     print(column)

columns = [pattern.search(column).group(0) for column in columns]
columns = [column.replace('^','') for column in columns]

column_names = columns
column_names
    
# columns[0] = 'Title'
# columns[-1] = 'Year'


['Title', 'Studio', 'Lifetime Gross', 'Year']

In [623]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 3)

%matplotlib inline

In [171]:
#first rough df for the domestic gross figures for movies
domestic_df = pd.DataFrame(list(movies.values()), columns = column_names)
domestic_df.head()

,Title,Studio,Lifetime Gross,Year
0,Star Wars: The Force Awakens,BV,"$936,662,225",2015
1,Avatar,Fox,"$760,507,625",2009^
2,Black Panther,BV,"$700,059,566",2018
3,Avengers: Infinity War,BV,"$678,815,482",2018
4,Titanic,Par.,"$659,363,944",1997^


In [374]:
# starting to scrape ski resort data
# grab data for each relevant ski resort: total snowfall, snow days, average/max base depth, average/max summit depth,
# biggest snowfall days
# injuries per year

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os
chromedriver = "/Users/dynogravelso/Documents/GitHub/luther/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [322]:
# go to base page
url = 'https://www.skiresort.info/ski-resorts/north-america/'
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [323]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [324]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
#navigate to ski lifts
# ski_lift_selector = '//a[contains(text(), "Ski lifts")]'
# driver.find_element_by_xpath(ski_lift_selector).click()

In [325]:
#find all resorts here
resort_soup = soup.find(id = 'resortList')
sub_list = resort_soup.find_all('a', {'class': 'h3'})    
resorts = [resort.text for resort in sub_list]
all_resorts = []
all_resorts += resorts

In [327]:
def scrape_once(url, all_resorts):
    driver.get(url)
    time.sleep(15)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    resort_soup = soup.find(id = 'resortList')
    sub_list = resort_soup.find_all('a', {'class': 'h3'})    
    resorts = [resort.text for resort in sub_list]
    all_resorts += resorts
    return all_resorts
    

In [328]:
#navigate to next page and find 
for i in range(2,6):
    url_new = 'https://www.skiresort.info/ski-resorts/north-america/page/' + str(i) + '/'
    all_resorts = scrape_once(url_new, all_resorts)

In [550]:
#clean up resort names
all_resorts = [resort.strip() for resort in all_resorts]
len(all_resorts)

4070

In [545]:
#opening the browser to scrape the names of all NA resorts
from collections import defaultdict
all_states = ['colorado','utah','wyoming','montana','idaho','new-mexico','arizona','california','lake-tahoe','nevada','oregon','washington','alaska','new-york','pennsylvania','vermont','new-hampshire','maine','massachusetts','michigan','minnesota','wisconsin','alberta','british-columbia','ontario','quebec']
url_list = ['https://www.onthesnow.com/' + state + '/skireport.html' for state in all_states]
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

caps = DesiredCapabilities().CHROME
# caps["pageLoadStrategy"] = "normal"  #  complete
# caps["pageLoadStrategy"] = "eager"  #  interactive
caps["pageLoadStrategy"] = "none"
driver = webdriver.Chrome(chromedriver, desired_capabilities=caps)

In [443]:
# grab all resort names and organize into dict
for i, url in enumerate(url_list):
    #grab all the resort names here and append to resort_list
    driver.get(url)
    time.sleep(10)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    resort_soup = soup.find_all('div', {'class': 'name link-light'})
    for resort in resort_soup:
        resort_dict[all_states[i]] = resort_dict[all_states[i]] + [resort.a.text]   

In [541]:
# change resort name dict to make ready for urls
import copy
resort_dict_urlf = resort_dict.copy()
for key in resort_dict_urlf:
    for i, value in enumerate(resort_dict_urlf[key]):
        resort_dict_urlf[key][i] = value.replace(' - ',' ').replace('.','').replace(' / ','-').replace("'","")
resort_dict_urlf['quebec'][0] = 'ski-bromont'
resort_dict_urlf['colorado'][-11] = 'durango-mountain-resort'
resort_dict_urlf['colorado'][-9] = 'ski-cooper'
print(resort_dict_urlf['utah'][5])
resort_dict_urlf['utah'][5] = 'wolf-creek'
print(resort_dict_urlf['utah'][6])
resort_dict_urlf['utah'][6] = 'park-city-mountain-resort'
print(resort_dict_urlf['wyoming'][2])
resort_dict_urlf['wyoming'][2] = 'hogadon'
print(resort_dict_urlf['wyoming'][4])
resort_dict_urlf['wyoming'][4] = 'snowy-range-ski-recreation-area'
resort_dict_urlf['montana'][-2] = 'showdown-ski-area'
resort_dict_urlf['montana'][-1] = 'teton-pass-ski-area'
resort_dict_urlf['new-mexico'][5] = 'sipapu-ski-and-summer-resort'
resort_dict_urlf['new-mexico'][-1] = 'enchanted-forest-cross-countryski-area'
resort_dict_urlf['california'][0] = 'badger-pass'
resort_dict_urlf['california'][3] = 'boreal'
resort_dict_urlf['california'][-10] = 'mount-shasta-board-ski-park'
resort_dict_urlf['california'][-3] = 'squaw-valley-usa'
resort_dict_urlf['nevada'][1] = 'las-vegas-ski-and-snowboard-resort'
resort_dict_urlf['oregon'][2] = 'mount-ashland'
resort_dict_urlf['oregon'][5] = 'mt-hood-ski-bowl'
resort_dict_urlf['oregon'][6] = 'mount-ashland'
resort_dict_urlf['new-york'][3] = 'catamount-ski-ride-area'
resort_dict_urlf['pennsylvania'][3] = 'blue-mountain-ski-area'
resort_dict_urlf['pennsylvania'][9] = 'sno-mountain'
resort_dict_urlf['pennsylvania'][-2] = 'big-bear'
resort_dict_urlf['vermont'][-6] = 'pico-mountain-at-killington'
resort_dict_urlf['michigan'][4] = 'blackjack-ski-resort'
resort_dict_urlf['michigan'][7] = 'caberfae-peaks-ski-golf-resort'
resort_dict_urlf['michigan'][10] = 'indianhead-mountain'
resort_dict_urlf['michigan'][12] = 'mount-brighton'
resort_dict_urlf['michigan'][13] = 'mount-holiday-ski-area'
resort_dict_urlf['michigan'][19] = 'shanty-creek'
resort_dict_urlf['minnesota'][5] = 'hyland-ski-snowboard-area'
resort_dict_urlf['minnesota'][-3] = 'wild-mountain-ski-snowboard-area'
resort_dict_urlf['minnesota'][-2] = 'coffee-mill-ski-snowboard-area'
resort_dict_urlf['minnesota'][-1] = 'elm-creek'
resort_dict_urlf['british-columbia'][6] = 'mount-seymour'
resort_dict_urlf['british-columbia'][10] = 'silver-star'
resort_dict_urlf['british-columbia'][13] = 'whitewater'
resort_dict_urlf['british-columbia'][-3] = 'revelstoke-mountain'
resort_dict_urlf['british-columbia'][-2] = 'hemlock-valley-resort'


resort_dict_urlf

wolf-creek
park-city-mountain-resort
hogadon
snowy-range-ski-recreation-area


defaultdict(list,
            {'colorado': ['Arapahoe Basin Ski Area',
              'Aspen-Snowmass',
              'Beaver Creek',
              'Breckenridge',
              'Copper Mountain Resort',
              'Crested Butte Mountain Resort',
              'Eldora Mountain Resort',
              'Howelsen Hill',
              'Keystone',
              'Loveland',
              'Monarch Mountain',
              'Powderhorn',
              'durango-mountain-resort',
              'Ski Granby Ranch',
              'ski-cooper',
              'Steamboat',
              'Sunlight Mountain Resort',
              'Telluride',
              'Vail',
              'Winter Park Resort',
              'Wolf Creek Ski Area',
              'Silverton Mountain',
              'Echo Mountain'],
             'utah': ['Alta Ski Area',
              'Beaver Mountain',
              'Brian Head Resort',
              'Brighton Resort',
              'Deer Valley Resort',
              'wolf-creek',

In [547]:
hist_snow_states = {}
for state in resort_dict_urlf:
    hist_snow_states[state] = {}
    for resort in resort_dict_urlf[state]:
        hist_snow_states[state][resort] = []
hist_snow_states

{'colorado': {'Arapahoe Basin Ski Area': [],
  'Aspen-Snowmass': [],
  'Beaver Creek': [],
  'Breckenridge': [],
  'Copper Mountain Resort': [],
  'Crested Butte Mountain Resort': [],
  'Eldora Mountain Resort': [],
  'Howelsen Hill': [],
  'Keystone': [],
  'Loveland': [],
  'Monarch Mountain': [],
  'Powderhorn': [],
  'durango-mountain-resort': [],
  'Ski Granby Ranch': [],
  'ski-cooper': [],
  'Steamboat': [],
  'Sunlight Mountain Resort': [],
  'Telluride': [],
  'Vail': [],
  'Winter Park Resort': [],
  'Wolf Creek Ski Area': [],
  'Silverton Mountain': [],
  'Echo Mountain': []},
 'utah': {'Alta Ski Area': [],
  'Beaver Mountain': [],
  'Brian Head Resort': [],
  'Brighton Resort': [],
  'Deer Valley Resort': [],
  'wolf-creek': [],
  'park-city-mountain-resort': [],
  'Powder Mountain': [],
  'Snowbasin': [],
  'Snowbird': [],
  'Solitude Mountain Resort': [],
  'Sundance': [],
  'Eagle Point': []},
 'wyoming': {'Snow King Resort': [],
  'Grand Targhee Resort': [],
  'hogadon'

In [539]:
#for each key and corresponding values in constructed resort dict go to the page and collect snow data ONLY FOR 2017
for i, state in enumerate(resort_dict_urlf):
    for j, resort in enumerate(resort_dict_urlf[state]):
        url = 'https://www.onthesnow.com/' + state + '/' + resort.lower().replace(' ','-') + '/historical-snowfall.html'
        driver.get(url)
        time.sleep(4)
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        soup = BeautifulSoup(driver.page_source, 'lxml')
        table_soup = soup.find('div', {'class': 'resBox hist_stats_main'})
        try:
            snow = table_soup.find_all('strong')
        except AttributeError:
            hist_snow_states[state][resort] = []
            print(state,resort,'fail')
            continue
        print(state,resort,'success')
#         print(hist_snow_states[state][resort])
        for s in snow:
            hist_snow_states[state][resort] = hist_snow_states[state][resort] + [s.text]    
    with open(str(state) + '.pkl', 'wb') as picklefile:
        pickle.dump(hist_snow_states[state], picklefile)

colorado Arapahoe Basin Ski Area success
colorado Aspen-Snowmass success
colorado Beaver Creek success
colorado Breckenridge success
colorado Copper Mountain Resort success
colorado Crested Butte Mountain Resort success
colorado Eldora Mountain Resort success
colorado Howelsen Hill success
colorado Keystone success
colorado Loveland success
colorado Monarch Mountain success
colorado Powderhorn success
colorado durango-mountain-resort success
colorado Ski Granby Ranch success
colorado ski-cooper success
colorado Steamboat success
colorado Sunlight Mountain Resort success
colorado Telluride success
colorado Vail success
colorado Winter Park Resort success
colorado Wolf Creek Ski Area success
colorado Silverton Mountain success
colorado Echo Mountain success
utah Alta Ski Area success
utah Beaver Mountain success
utah Brian Head Resort success
utah Brighton Resort success
utah Deer Valley Resort success
utah Nordic Valley Resort fail
utah Park City fail
utah Powder Mountain success
utah S

massachusetts Bousquet Ski Area success
massachusetts Bradford Ski Area success
massachusetts Ski Butternut success
massachusetts Jiminy Peak success
massachusetts Nashoba Valley success
massachusetts Otis Ridge Ski Area success
massachusetts Wachusett Mountain Ski Area success
massachusetts Blue Hills Ski Area success
massachusetts Ski Ward success
michigan Alpine Valley Ski Area success
michigan Apple Mountain success
michigan Big Powderhorn Mountain success
michigan Bittersweet Ski Area success
michigan Big Snow Resort Blackjack fail
michigan Boyne Highlands success
michigan Boyne Mountain Resort success
michigan Caberfae Peaks fail
michigan Cannonsburg success
michigan Crystal Mountain fail
michigan Big Snow Resort Indianhead Mountain fail
michigan Marquette Mountain success
michigan Mt Brighton fail
michigan Mt Holiday Ski Area fail
michigan Mount Holly success
michigan Mont Ripley success
michigan Norway Mountain success
michigan Nubs Nob Ski Area success
michigan Pine Mountain s

{'colorado': {'Arapahoe Basin Ski Area': ['208 "',
   '64',
   '73 "',
   '42 "',
   '73 "',
   '42 "',
   '14 "'],
  'Aspen-Snowmass': ['233 "', '42', '52 "', '29 "', '80 "', '50 "', '16 "'],
  'Beaver Creek': ['193 "', '43', '68 "', '33 "', '68 "', '33 "', '12 "'],
  'Breckenridge': ['271 "', '62', '83 "', '42 "', '83 "', '42 "', '14 "'],
  'Copper Mountain Resort': ['166 "',
   '50',
   '73 "',
   '41 "',
   '98 "',
   '55 "',
   '18 "'],
  'Crested Butte Mountain Resort': ['145 "',
   '40',
   '57 "',
   '33 "',
   '57 "',
   '33 "',
   '11 "'],
  'Eldora Mountain Resort': ['192 "',
   '55',
   '40 "',
   '33 "',
   '40 "',
   '33 "',
   '17 "'],
  'Howelsen Hill': ['93 "', '27', '33 "', '16 "', '33 "', '16 "', '12 "'],
  'Keystone': ['154 "', '44', '74 "', '45 "', '74 "', '45 "', '15 "'],
  'Loveland': ['265 "', '71', '79 "', '45 "', '79 "', '45 "', '14 "'],
  'Monarch Mountain': ['167 "', '47', '68 "', '41 "', '68 "', '41 "', '14 "'],
  'Powderhorn': ['122 "', '26', '44 "', '24 "

In [548]:
# Collect historical snow data for years 2016,2015,2014,2013
for year in ['2016','2015','2014','2013']:
    for i, state in enumerate(resort_dict_urlf):
        for j, resort in enumerate(resort_dict_urlf[state]):
            url = 'https://www.onthesnow.com/' + state + '/' + resort.lower().replace(' ','-') + '/historical-snowfall.html' + '?&y=' + year
            driver.get(url)
            time.sleep(4)
    #         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            soup = BeautifulSoup(driver.page_source, 'lxml')
            table_soup = soup.find('div', {'class': 'resBox hist_stats_main'})
            try:
                snow = table_soup.find_all('strong')
            except AttributeError:
                hist_snow_states[state][resort] = []
                print(state,resort,'fail')
                continue
            print(state,resort,'success')
    #         print(hist_snow_states[state][resort])
            for s in snow:
                hist_snow_states[state][resort] = hist_snow_states[state][resort] + [s.text] 
            if year == '2016':
                stat = table_soup.find_all('span')
                for s in stat:
                    hist_snow_states[state][resort] = [s.text] + hist_snow_states[state][resort]
        with open(str(state) + year + '.pkl', 'wb') as picklefile:
            pickle.dump(hist_snow_states[state], picklefile)

colorado Arapahoe Basin Ski Area success
colorado Aspen-Snowmass success
colorado Beaver Creek success
colorado Breckenridge success
colorado Copper Mountain Resort success
colorado Crested Butte Mountain Resort success
colorado Eldora Mountain Resort success
colorado Howelsen Hill success
colorado Keystone success
colorado Loveland success
colorado Monarch Mountain success
colorado Powderhorn success
colorado durango-mountain-resort success
colorado Ski Granby Ranch success
colorado ski-cooper success
colorado Steamboat success
colorado Sunlight Mountain Resort success
colorado Telluride success
colorado Vail success
colorado Winter Park Resort success
colorado Wolf Creek Ski Area success
colorado Silverton Mountain success
colorado Echo Mountain success
utah Alta Ski Area success
utah Beaver Mountain success
utah Brian Head Resort success
utah Brighton Resort success
utah Deer Valley Resort success
utah wolf-creek success
utah park-city-mountain-resort success
utah Powder Mountain su

maine Big Squaw Mountain Ski Resort success
massachusetts Berkshire East success
massachusetts Blandford Ski Area success
massachusetts Bousquet Ski Area success
massachusetts Bradford Ski Area success
massachusetts Ski Butternut success
massachusetts Jiminy Peak success
massachusetts Nashoba Valley success
massachusetts Otis Ridge Ski Area success
massachusetts Wachusett Mountain Ski Area success
massachusetts Blue Hills Ski Area success
massachusetts Ski Ward success
michigan Alpine Valley Ski Area success
michigan Apple Mountain success
michigan Big Powderhorn Mountain success
michigan Bittersweet Ski Area success
michigan blackjack-ski-resort success
michigan Boyne Highlands success
michigan Boyne Mountain Resort success
michigan caberfae-peaks-ski-golf-resort success
michigan Cannonsburg success
michigan Crystal Mountain success
michigan indianhead-mountain success
michigan Marquette Mountain success
michigan mount-brighton success
michigan mount-holiday-ski-area success
michigan 

lake-tahoe Tahoe Donner fail
nevada Diamond Peak success
nevada las-vegas-ski-and-snowboard-resort success
nevada Mt Rose Ski Tahoe success
nevada Elko SnoBowl fail
oregon Anthony Lakes Mountain Resort success
oregon Hoodoo Ski Area success
oregon mount-ashland success
oregon Mt Bachelor success
oregon Mt Hood Meadows success
oregon mt-hood-ski-bowl fail
oregon mount-ashland success
oregon Willamette Pass success
oregon Cooper Spur success
oregon Spout Springs fail
washington 49 Degrees North success
washington Bluewood success
washington Crystal Mountain success
washington Mission Ridge success
washington Mt Baker success
washington Mt Spokane Ski and Snowboard Park success
washington Stevens Pass Resort success
washington The Summit at Snoqualmie success
washington White Pass fail
washington Alpental success
alaska Alyeska Resort success
alaska Eaglecrest Ski Area success
alaska Hilltop Ski Area success
new-york Belleayre success
new-york Brantling Ski Slopes success
new-york Bristol

quebec Le Massif success
quebec Mont Blanc success
quebec Mont Orford success
quebec Mont Saint-Sauveur success
quebec Mont Sutton success
quebec Mont Sainte Anne success
quebec Mont Cascades success
quebec Owls Head success
quebec Camp Fortune success
quebec Mont Ste-Marie success
quebec Stoneham success
quebec Tremblant success
quebec Vorlage success
quebec Massif du Sud success
colorado Arapahoe Basin Ski Area success
colorado Aspen-Snowmass success
colorado Beaver Creek success
colorado Breckenridge success
colorado Copper Mountain Resort success
colorado Crested Butte Mountain Resort success
colorado Eldora Mountain Resort success
colorado Howelsen Hill success
colorado Keystone success
colorado Loveland success
colorado Monarch Mountain success
colorado Powderhorn success
colorado durango-mountain-resort success
colorado Ski Granby Ranch success
colorado ski-cooper success
colorado Steamboat success
colorado Sunlight Mountain Resort success
colorado Telluride success
colorado Vai

new-hampshire Granite Gorge success
new-hampshire Whaleback Mountain success
maine Camden Snow Bowl success
maine Lost Valley success
maine Mt Abram Ski Resort success
maine Mt Jefferson success
maine New Hermon Mountain success
maine Saddleback Inc fail
maine Shawnee Peak success
maine Sugarloaf success
maine Sunday River success
maine Big Squaw Mountain Ski Resort success
massachusetts Berkshire East success
massachusetts Blandford Ski Area success
massachusetts Bousquet Ski Area success
massachusetts Bradford Ski Area success
massachusetts Ski Butternut success
massachusetts Jiminy Peak success
massachusetts Nashoba Valley success
massachusetts Otis Ridge Ski Area success
massachusetts Wachusett Mountain Ski Area success
massachusetts Blue Hills Ski Area success
massachusetts Ski Ward success
michigan Alpine Valley Ski Area success
michigan Apple Mountain success
michigan Big Powderhorn Mountain success
michigan Bittersweet Ski Area success
michigan blackjack-ski-resort success
mich

lake-tahoe Heavenly Mountain Resort fail
lake-tahoe Homewood Mountain Resort fail
lake-tahoe Kirkwood fail
lake-tahoe Mt Rose Ski Tahoe fail
lake-tahoe Northstar California fail
lake-tahoe Sierra-at-Tahoe fail
lake-tahoe Soda Springs fail
lake-tahoe Squaw Valley Alpine Meadows fail
lake-tahoe Sugar Bowl Resort fail
lake-tahoe Tahoe Donner fail
nevada Diamond Peak success
nevada las-vegas-ski-and-snowboard-resort success
nevada Mt Rose Ski Tahoe success
nevada Elko SnoBowl success
oregon Anthony Lakes Mountain Resort success
oregon Hoodoo Ski Area success
oregon mount-ashland success
oregon Mt Bachelor success
oregon Mt Hood Meadows success
oregon mt-hood-ski-bowl success
oregon mount-ashland success
oregon Willamette Pass success
oregon Cooper Spur success
oregon Spout Springs success
washington 49 Degrees North success
washington Bluewood success
washington Crystal Mountain fail
washington Mission Ridge success
washington Mt Baker success
washington Mt Spokane Ski and Snowboard Park s

ontario Blue Mountain success
ontario Calabogie Peaks success
ontario Loch Lomond success
ontario Mt Pakenham success
ontario Searchmont Resort success
ontario Horseshoe Resort success
ontario Mt St Louis-Moonstone success
ontario Hidden Valley success
ontario Glen Eden success
quebec ski-bromont success
quebec Edelweiss Valley success
quebec Le Massif success
quebec Mont Blanc success
quebec Mont Orford success
quebec Mont Saint-Sauveur success
quebec Mont Sutton fail
quebec Mont Sainte Anne success
quebec Mont Cascades fail
quebec Owls Head success
quebec Camp Fortune success
quebec Mont Ste-Marie success
quebec Stoneham success
quebec Tremblant success
quebec Vorlage success
quebec Massif du Sud success
colorado Arapahoe Basin Ski Area success
colorado Aspen-Snowmass success
colorado Beaver Creek success
colorado Breckenridge success
colorado Copper Mountain Resort success
colorado Crested Butte Mountain Resort fail
colorado Eldora Mountain Resort success
colorado Howelsen Hill succ

new-hampshire King Pine success
new-hampshire Loon Mountain success
new-hampshire Mount Sunapee success
new-hampshire Pats Peak success
new-hampshire Ragged Mountain Resort success
new-hampshire Tenney Mountain fail
new-hampshire Waterville Valley success
new-hampshire Wildcat Mountain success
new-hampshire Crotched Mountain success
new-hampshire Granite Gorge success
new-hampshire Whaleback Mountain success
maine Camden Snow Bowl success
maine Lost Valley success
maine Mt Abram Ski Resort success
maine Mt Jefferson success
maine New Hermon Mountain success
maine Saddleback Inc success
maine Shawnee Peak success
maine Sugarloaf success
maine Sunday River success
maine Big Squaw Mountain Ski Resort success
massachusetts Berkshire East success
massachusetts Blandford Ski Area success
massachusetts Bousquet Ski Area success
massachusetts Bradford Ski Area success
massachusetts Ski Butternut success
massachusetts Jiminy Peak success
massachusetts Nashoba Valley success
massachusetts Otis R

In [551]:
hist_snow_states

{'colorado': {'Arapahoe Basin Ski Area': ['Biggest Snowfall:',
   'Average Summit Depth:',
   'Max Summit Depth:',
   'Average Base Depth:',
   'Max Base Depth:',
   'Total Snowfall Days:',
   'Total Snowfall:',
   '208 "',
   '64',
   '73 "',
   '42 "',
   '73 "',
   '42 "',
   '14 "',
   '305 "',
   '82',
   '78 "',
   '51 "',
   '78 "',
   '50 "',
   '16 "',
   '334 "',
   '93',
   '80 "',
   '49 "',
   '80 "',
   '49 "',
   '18 "',
   '311 "',
   '99',
   '62 "',
   '39 "',
   '62 "',
   '39 "',
   '12 "',
   '451 "',
   '123',
   '90 "',
   '53 "',
   '90 "',
   '53 "',
   '19 "'],
  'Aspen-Snowmass': ['Biggest Snowfall:',
   'Average Summit Depth:',
   'Max Summit Depth:',
   'Average Base Depth:',
   'Max Base Depth:',
   'Total Snowfall Days:',
   'Total Snowfall:',
   '233 "',
   '42',
   '52 "',
   '29 "',
   '80 "',
   '50 "',
   '16 "',
   '319 "',
   '69',
   '4545 "',
   '67 "',
   '100 "',
   '65 "',
   '19 "',
   '305 "',
   '62',
   '72 "',
   '38 "',
   '99 "',
   '59

In [664]:
# making states column
states_for_df = []
resorts_for_df = []
for state in hist_snow_states:
    states_for_df = states_for_df + [state for resort_num in range(0, len(state))]
    for resort in hist_snow_states[state]:
        resorts_for_df.append(resort)

['Arapahoe Basin Ski Area',
 'Aspen-Snowmass',
 'Beaver Creek',
 'Breckenridge',
 'Copper Mountain Resort',
 'Crested Butte Mountain Resort',
 'Eldora Mountain Resort',
 'Howelsen Hill',
 'Keystone',
 'Loveland',
 'Monarch Mountain',
 'Powderhorn',
 'durango-mountain-resort',
 'Ski Granby Ranch',
 'ski-cooper',
 'Steamboat',
 'Sunlight Mountain Resort',
 'Telluride',
 'Vail',
 'Winter Park Resort',
 'Wolf Creek Ski Area',
 'Silverton Mountain',
 'Echo Mountain',
 'Alta Ski Area',
 'Beaver Mountain',
 'Brian Head Resort',
 'Brighton Resort',
 'Deer Valley Resort',
 'wolf-creek',
 'park-city-mountain-resort',
 'Powder Mountain',
 'Snowbasin',
 'Snowbird',
 'Solitude Mountain Resort',
 'Sundance',
 'Eagle Point',
 'Snow King Resort',
 'Grand Targhee Resort',
 'hogadon',
 'Jackson Hole',
 'snowy-range-ski-recreation-area',
 'White Pine Ski Area',
 'Sleeping Giant Ski Resort',
 'Meadowlark Ski Lodge',
 'Whitefish Mountain Resort',
 'Big Sky Resort',
 'Blacktail Mountain Ski Area',
 'Bridger

In [690]:
#cleaning up dict to prepare for input into columns to input into df
hist_snow_states_avg = hist_snow_states.copy()

for state in hist_snow_states_copy:
    for resort in hist_snow_states_copy[state]:
        counter = 0
        for i, stat in enumerate(hist_snow_states_copy[state][resort]):
            stat_str = stat.copy()
            try:
                stat = int(stat.replace('"','').strip())
            except:
                pass
            if isinstance(stat, str):
                counter += 1       
            else:
                stat = stat_str
                break
        avg_stats = [0 for i in range(0, counter)]
        for i, stat in enumerate(hist_snow_states_copy[state][resort][counter:]):
            stat = int(stat.replace('"','').strip())
            avg_stats[i%counter] += stat
        map((lambda x: x/counter), avg_stats)
        hist_snow_states_copy[state][resort] = hist_snow_states_copy[state][resort][:counter] + avg_stats
        break
# hist_snow_states_copy

AttributeError: 'str' object has no attribute 'copy'

In [677]:
biggest_snowfall = []
average_summit_depth = []
max_summit_depth = []
average_base_depth = []
max_base_depth = []
total_snowfall_days = []
total_snowfall = []

stat_dict = {'Biggest Snowfall:' : biggest_snowfall, 'Average Summit Depth:' : average_summit_depth, 'Max Summit Depth:' : max_summit_depth, 'Average Base Depth:' : average_base_depth, 'Max Base Depth:' : max_base_depth, 'Total Snowfall Days:' : total_snowfall_days, 'Total Snowfall:' : total_snowfall}

all_stats = [biggest_snowfall, average_summit_depth, max_summit_depth, average_base_depth, max_base_depth, total_snowfall_days, total_snowfall]

hist_snow_states_avg = hist_snow_states.copy()

for state in hist_snow_states_copy:
    for resort in hist_snow_states_copy[state]:
        for i, stat in enumerate(hist_snow_states_copy[state][resort]):
            stat_names = []
            if isinstance(stat, str):
                stat_names.append(stat)                
            else:
                print(stat)
                stat_dict[stat_name[i % len(stat_names)]] = stat_dict[stat_name[i % len(stat_names)]] + [stat]
                
# stat_dict              
                

In [672]:
# making resort column
test = 1
type(test) == 'int'


False

In [471]:
snow_report_columns = ['Total Snowfall', 'Total Snowfall Days', 'Max Base Depth', 'Average Base Depth', 'Max Summit Depth', 'Average Summit Depth', 'Biggest Snowfall']

In [587]:
ski_df = pd.read_csv('skiResort.csv', encoding = 'latin-1')
ski_df.columns = [column.strip() for column in ski_df.columns]
ski_NA_df = ski_df[ski_df.Continent == 'North America']

In [588]:
# ski_USA_df = ski_USA_df['Resort Name', 'State/Province', 'Snow reliability', 'Adult', ]
ski_NA_df_test = ski_NA_df.copy()
ski_NA_df_test.head()

,Resort Name,Continent,Country,State/Province,URL,Altitude,Easy,Intermediate,Difficult,Funicular,Circulating ropeway/gondola lift,Chairlift,T-bar lift/platter/button lift,Sunkid Moving Carpet,Adult,Youth,Child,Currency,Ski resort size,"Slope offering, variety of runs",Lifts and cable cars,Snow reliability,Slope preparation,"Access, on-site parking","Orientation (trail map, information boards, sign-postings)",Cleanliness and hygiene,Environmentally friendly ski operation,Friendliness of staff,"Mountain restaurants, ski huts, gastronomy",Après-ski,Accommodation offering directly at the slopes and lifts,Families and children,Beginners,"Advanced skiers, freeriders",Snow parks,Cross-country skiing and trails,Aerial tramway/reversible ropeway,Rope tow/beginner lift,People mover,Combined installation (gondola and chair),Cog railway,Helicopter for Heli-skiing,Snow caterpillars for Cat-skiing
2,49-degrees-north-mountain-resort,North America,USA,Washington State,http://www.skiresort.info/ski-resort/49-degree...,1760.0,20.0,27.0,21.0,NaN,NaN,0.0,NaN,NaN,58.0,0.0,48.0,US Dollar,3.5,3.9,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
14,abenaki,North America,USA,New Hampshire,http://www.skiresort.info/ski-resort/abenaki/,250.0,0.5,0.3,0.2,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,aberdeen-heli-prince-george,North America,Canada,British Columbia,http://www.skiresort.info/ski-resort/aberdeen-...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
24,adamants,North America,Canada,British Columbia,http://www.skiresort.info/ski-resort/adamants/,3100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
25,adanac-ski-hill,North America,Canada,Ontario,http://www.skiresort.info/ski-resort/adanac-sk...,340.0,1.3,0.6,0.1,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [597]:
drop_columns = ['Circulating ropeway/gondola lift', 'URL', 'Continent', 'Funicular', 'Chairlift', 'T-bar lift/platter/button lift', 'Sunkid Moving Carpet', 'Currency']
return
ski_NA_df_test = ski_NA_df.drop(columns = drop_columns)
ski_NA_df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 822 entries, 2 to 5376
Data columns (total 35 columns):
Resort Name                                                   822 non-null object
Country                                                       822 non-null object
State/Province                                                810 non-null object
Altitude                                                      822 non-null float64
Easy                                                          680 non-null float64
Intermediate                                                  680 non-null float64
Difficult                                                     680 non-null float64
Adult                                                         822 non-null float64
Youth                                                         822 non-null float64
Child                                                         822 non-null float64
Ski resort size                                               474 n

In [600]:
#dropping columns with too much missing data
return
for column in ski_NA_df_test:
    if ski_NA_df_test[column].count() < 400:
        ski_NA_df_test.drop(columns = [column], inplace = True)

In [631]:
#cleaning up column names
for i in ski_NA_df_test:
    new_name = i.replace(' ', '_').replace(',', '').replace('/Province','')
    ski_NA_df_test.rename(columns = {i : new_name}, inplace = True)

ski_NA_df_test.rename(columns = {'Slope_offering_variety_of_runs' : 'Run_variety'}, inplace = True)
ski_NA_df_test.rename(columns = {'Lifts_and_cable_cars' : 'Lifts'}, inplace = True)
ski_NA_df_test.rename(columns = {'Ski_resort_size' : 'Resort_size'}, inplace = True)

ski_NA_df_test.head()

,Resort_Name,Country,State,Altitude,Easy,Intermediate,Difficult,Adult,Youth,Child,Resort_size,Run_variety,Lifts
2,49-degrees-north-mountain-resort,USA,Washington State,1760.0,20.0,27.0,21.0,58.0,0.0,48.0,3.5,3.9,3.0
14,abenaki,USA,New Hampshire,250.0,0.5,0.3,0.2,0.0,0.0,0.0,NaN,NaN,NaN
15,aberdeen-heli-prince-george,Canada,British Columbia,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
24,adamants,Canada,British Columbia,3100.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
25,adanac-ski-hill,Canada,Ontario,340.0,1.3,0.6,0.1,0.0,0.0,0.0,NaN,NaN,NaN


In [628]:
ski_NA_df_clean1 = ski_NA_df_test

In [635]:
ski_NA_df_clean1.dropna(subset = ['Resort_size', 'Run_variety', 'Lifts'], inplace = True)

,Resort_Name,Country,State,Altitude,Easy,Intermediate,Difficult,Adult,Youth,Child,Resort_size,Run_variety,Lifts
2,49-degrees-north-mountain-resort,USA,Washington State,1760.0,20.0,27.0,21.0,58.00,0.00,48.00,3.5,3.9,3.0
34,afton-alps,USA,Minnesota,387.0,3.0,3.0,2.0,58.00,0.00,53.00,1.9,2.3,3.0
80,alice-hill-park-pembroke,Canada,Ontario,210.0,1.0,0.5,0.0,22.00,0.00,18.00,1.3,1.3,1.5
101,alpental,USA,Washington State,1670.0,0.5,2.3,6.8,79.00,55.00,15.00,2.1,2.3,3.5
105,alpine-meadows,USA,Western United States,2632.0,25.0,40.0,35.0,159.00,134.00,98.00,3.0,5.0,4.0
106,alpine-mountain-ski-snow-tubing-center,USA,Pennsylvania,351.0,2.0,4.0,2.0,55.00,0.00,45.00,1.9,2.3,3.0
107,alpine-ski-club-collingwood,Canada,Ontario,460.0,10.0,6.0,4.0,80.00,0.00,55.00,2.4,2.7,3.4
109,alpine-valley-cleveland,USA,Ohio,457.0,0.4,0.6,0.5,43.00,0.00,38.00,1.3,1.7,2.9
110,alpine-valley-resort-elkhorn,USA,Wisconsin,415.0,1.5,1.5,1.0,48.00,0.00,43.00,1.5,1.9,3.5
119,alta,USA,Utah,3215.0,29.0,46.4,40.6,104.00,0.00,54.00,4.3,4.7,3.7


In [666]:
ski_NA_df_clean1.dropna(subset = ['State'], inplace = True)
ski_NA_df_clean1.dropna(inplace = True)
ski_NA_df_test.rename(columns = {'Resort_Name' : 'Resort_name'}, inplace = True)

In [667]:
ski_NA_df_clean1.sort_values(by = 'Resort_name')

,Resort_name,Country,State,Altitude,Easy,Intermediate,Difficult,Adult,Youth,Child,Resort_size,Run_variety,Lifts
2,49-degrees-north-mountain-resort,USA,Washington State,1760.0,20.0,27.0,21.0,58.00,0.00,48.00,3.5,3.9,3.0
34,afton-alps,USA,Minnesota,387.0,3.0,3.0,2.0,58.00,0.00,53.00,1.9,2.3,3.0
80,alice-hill-park-pembroke,Canada,Ontario,210.0,1.0,0.5,0.0,22.00,0.00,18.00,1.3,1.3,1.5
101,alpental,USA,Washington State,1670.0,0.5,2.3,6.8,79.00,55.00,15.00,2.1,2.3,3.5
105,alpine-meadows,USA,Western United States,2632.0,25.0,40.0,35.0,159.00,134.00,98.00,3.0,5.0,4.0
106,alpine-mountain-ski-snow-tubing-center,USA,Pennsylvania,351.0,2.0,4.0,2.0,55.00,0.00,45.00,1.9,2.3,3.0
107,alpine-ski-club-collingwood,Canada,Ontario,460.0,10.0,6.0,4.0,80.00,0.00,55.00,2.4,2.7,3.4
109,alpine-valley-cleveland,USA,Ohio,457.0,0.4,0.6,0.5,43.00,0.00,38.00,1.3,1.7,2.9
110,alpine-valley-resort-elkhorn,USA,Wisconsin,415.0,1.5,1.5,1.0,48.00,0.00,43.00,1.5,1.9,3.5
119,alta,USA,Utah,3215.0,29.0,46.4,40.6,104.00,0.00,54.00,4.3,4.7,3.7


In [668]:
#make pkl to save
ski_NA_df_clean1.to_pickle('csv_df.pkl')

If time permits, could've gotten more features:
proximity to largest neighboring city
snow park rating
ski village rating

Resorts I've been to:
Whistler 4.7
Heavenly 4
Kirkwood 4
Boyne Highland
Squaw Valley
Treetops
Chamonix

Ski resorts  
Everyone wants: cheap
I want: good snow and has enough non-beginner slopes
good? means average base thickness and high snow days. regress between those two and snow reliability rating?
Friends want: (beginner slopes exist)
Maybe create a formula that balances price, slopes and thickness. With friends: more weight on price. By myself: less weight on price
Really for me it's about having as few bad days as possible while paying less
(Not much regression required)
Also: Can find the relationship between (snow, number of trails other factors, proximity to nearest airport or big city) and price. Regression required, but what problem am I trying to solve?

Could find the correlation between ratings for a season with traffic, snowfall and other factors affecting ratings

Plane tickets  
Plane tickets for two weeks in HK
Problems: When to visit and when to buy? Single trip or round trip. Stop or no stop.

Process:
scrape data and find csvs (selenium and beautifulsoup)
clean with modules then manually clean (patsy)
use linear regression on applicable data series (scikitlearn)
make graphs(seaborn)

In [447]:
import pickle

with open('all_resorts.pkl', 'wb') as picklefile:
    pickle.dump(resort_dict, picklefile)

In [482]:
with open("all_resorts.pkl", 'rb') as picklefile: 
    resort_dict = pickle.load(picklefile)

In [385]:
driver.close()

In [ ]:
# start scraping airplane ticket prices